In [1]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as fn
from pyspark.sql import SparkSession
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
spark = (SparkSession.builder
         .config("spark.jars","""/home/jovyan/jars/aws-java-sdk-core-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-dynamodb-1.11.534.jar,
                                 /home/jovyan/jars/aws-java-sdk-s3-1.11.534.jar,
                                 /home/jovyan/jars/hadoop-aws-3.2.2.jar""")
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "aulafia")
         .config("spark.hadoop.fs.s3a.secret.key", "aulafia@123")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
         .getOrCreate()
        )

In [4]:
df_municipio = spark.read.format('parquet').load('s3a://bronze/ibge/municipio')
df_municipio.dtypes

[('id', 'int'),
 ('nome', 'string'),
 ('microrregiao',
  'struct<id:int,nome:string,mesorregiao:struct<id:int,nome:string,UF:struct<id:int,sigla:string,nome:string,regiao:struct<id:int,sigla:string,nome:string>>>>'),
 ('regiao-imediata',
  'struct<id:int,nome:string,regiao-intermediaria:struct<id:int,nome:string,UF:struct<id:int,sigla:string,nome:string,regiao:struct<id:int,sigla:string,nome:string>>>>')]

In [5]:
df_municipio.show(12, False)

+-------+---------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|id     |nome                 |microrregiao                                                                            |regiao-imediata                                                              |
+-------+---------------------+----------------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|1100015|Alta Floresta D'Oeste|{11006, Cacoal, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}           |{110005, Cacoal, {1102, Ji-Paraná, {11, RO, Rondônia, {1, N, Norte}}}}       |
|1100023|Ariquemes            |{11003, Ariquemes, {1102, Leste Rondoniense, {11, RO, Rondônia, {1, N, Norte}}}}        |{110002, Ariquemes, {1101, Porto Velho, {11, RO, Rondônia, {1, N, Norte}}}}  |
|1100

In [6]:
df_municipio_campos = (
   df_municipio.select(fn.col("id"),
                       fn.col("nome"),
                       fn.col("microrregiao.id").alias('microregiao_id'),
                       fn.col("microrregiao.nome").alias('microregiao_nome'),
                       fn.col("microrregiao.mesorregiao.id").alias('microregiao_mesoregiao_id'),
                       fn.col("microrregiao.mesorregiao.nome").alias('microregiao_mesoregiao_nome'),
                       fn.col("microrregiao.mesorregiao.UF.id").alias('microregiao_mesoregiao_uf_id'),
                       fn.col("microrregiao.mesorregiao.UF.sigla").alias('microregiao_mesoregiao_uf_sigla'),
                       fn.col("microrregiao.mesorregiao.UF.nome").alias('microregiao_mesoregiao_uf_nome'),
                       fn.col("microrregiao.mesorregiao.UF.regiao.id").alias('microregiao_mesoregiao_uf_regiao_id'),
                       fn.col("microrregiao.mesorregiao.UF.regiao.sigla").alias('microregiao_mesoregiao_uf_regiao_sigla'),
                       fn.col("microrregiao.mesorregiao.UF.regiao.nome").alias('microregiao_mesoregiao_uf_regiao_nome'),
                       fn.col("regiao-imediata.id").alias('regiao_imediata_id'),
                       fn.col("regiao-imediata.nome").alias('regiao_imediata_nome'),
                       fn.col("regiao-imediata.regiao-intermediaria.id").alias('regiao_imediata_intermediaria_id'),
                       fn.col("regiao-imediata.regiao-intermediaria.nome").alias('regiao_imediata_intermediaria_nome'),
                       fn.col("regiao-imediata.regiao-intermediaria.UF.id").alias('regiao_imediata_intermediaria_uf_id'),
                       fn.col("regiao-imediata.regiao-intermediaria.UF.sigla").alias('regiao_imediata_intermediaria_uf_sigla'),
                       fn.col("regiao-imediata.regiao-intermediaria.UF.nome").alias('regiao_imediata_intermediaria_uf_nome'),
                       fn.col("regiao-imediata.regiao-intermediaria.UF.regiao.id").alias('regiao_imediata_intermediaria_uf_regiao_id'),
                       fn.col("regiao-imediata.regiao-intermediaria.UF.regiao.sigla").alias('regiao_imediata_intermediaria_uf_regiao_sigla'),
                       fn.col("regiao-imediata.regiao-intermediaria.UF.regiao.nome").alias('regiao_imediata_intermediaria_uf_regiao_nome')
                      )
                    )

In [7]:
df_municipio_campos.show(12, False)

+-------+---------------------+--------------+-----------------+-------------------------+---------------------------+----------------------------+-------------------------------+------------------------------+-----------------------------------+--------------------------------------+-------------------------------------+------------------+--------------------+--------------------------------+----------------------------------+-----------------------------------+--------------------------------------+-------------------------------------+------------------------------------------+---------------------------------------------+--------------------------------------------+
|id     |nome                 |microregiao_id|microregiao_nome |microregiao_mesoregiao_id|microregiao_mesoregiao_nome|microregiao_mesoregiao_uf_id|microregiao_mesoregiao_uf_sigla|microregiao_mesoregiao_uf_nome|microregiao_mesoregiao_uf_regiao_id|microregiao_mesoregiao_uf_regiao_sigla|microregiao_mesoregiao_uf_regiao_nom

In [8]:
(df_municipio_campos
 .write
 .format('parquet')
 .mode('overwrite')
 .save('s3a://silver/dado_geografico/municipio')
 )

In [9]:
pandasDF = df_municipio_campos.toPandas()

In [11]:
pandasDF.to_excel('api_ibge_municipios_export.xlsx', sheet_name='municipios', index = False)